**Implementing a Neural Net**

In [1]:
import pandas as pd
from sklearn.neural_network import MLPClassifier

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_extraction import DictVectorizer

from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import numpy as np

In [2]:
from google.colab import files
uploaded = files.upload()


Saving to_neural_net.csv to to_neural_net.csv


In [65]:
import io
face_recog = pd.read_csv(io.BytesIO(uploaded['to_neural_net.csv']))

In [66]:
face_recog['age_bins'] = pd.cut(x=face_recog['age'], 
                                bins=[0, 9, 29, 39, 49, 59, 69, 79, 89, 99, 130])
face_recog['age_encoded'] = pd.cut(x=face_recog['age'], 
                                bins=[0, 9, 29, 39, 49, 59, 69, 79, 89, 99, 130], 
                                labels = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
face_recog.head()

,age,ethnicity,gender,path,landmarks,nose2lip,lip2chin,len_between_eyes,left_eyelid_len,right_eyelid_len,nose_width,size_of_mouth,nose_len,bottom_lip_width,left_eye_size,right_eye_size,age_bins,age_encoded
0,1,Asian,Male,images\img0.jpg,"[[-7, 51], [-5, 81], [-1, 110], [5, 136], [18,...",0.127843,0.172767,0.237232,0.187906,0.179991,0.153473,0.274431,0.283629,0.060333,0.145301,0.109268,"(0, 9]",0
1,1,Asian,Male,images\img1.jpg,"[[6, 65], [10, 93], [17, 119], [24, 144], [35,...",0.115510,0.146110,0.275589,0.212916,0.201710,0.155617,0.248926,0.263344,0.078643,0.122885,0.084790,"(0, 9]",0
2,1,Asian,Male,images\img2.jpg,"[[11, 75], [12, 102], [17, 129], [23, 153], [3...",0.093780,0.214220,0.256911,0.193784,0.204183,0.151439,0.279955,0.264790,0.077230,0.121487,0.094426,"(0, 9]",0
3,1,Asian,Male,images\img3.jpg,"[[-6, 78], [-4, 105], [-1, 131], [3, 156], [13...",0.087586,0.221280,0.266966,0.170970,0.172216,0.181050,0.357491,0.256329,0.069628,0.121071,0.074953,"(0, 9]",0
4,1,Asian,Male,images\img4.jpg,"[[-4, 79], [-2, 104], [-1, 129], [0, 155], [8,...",0.093075,0.274444,0.275523,0.153957,0.150599,0.191064,0.422118,0.252346,0.054845,0.095940,0.049664,"(0, 9]",0


In [67]:
face_recog[(face_recog["age"] <= 25) &(face_recog["age"]>18)]

,age,ethnicity,gender,path,landmarks,nose2lip,lip2chin,len_between_eyes,left_eyelid_len,right_eyelid_len,nose_width,size_of_mouth,nose_len,bottom_lip_width,left_eye_size,right_eye_size,age_bins,age_encoded
2462,19,Black,Female,images\img2588.jpg,"[[-24, 66], [-23, 95], [-20, 125], [-13, 154],...",0.067778,0.211139,0.244353,0.164625,0.160613,0.193207,0.355267,0.266359,0.091054,0.126211,0.088633,"(9, 29]",1
2463,19,Black,Male,images\img2589.jpg,"[[18, 74], [19, 98], [21, 123], [28, 146], [37...",0.110546,0.227716,0.269462,0.160917,0.157808,0.210588,0.402552,0.281605,0.103447,0.089772,0.057470,"(9, 29]",1
2464,19,Asian,Male,images\img2590.jpg,"[[20, 69], [22, 93], [25, 116], [27, 140], [32...",0.141747,0.257801,0.291191,0.173785,0.162585,0.175814,0.379138,0.285756,0.061634,0.088592,0.050738,"(9, 29]",1
2465,19,Asian,Male,images\img2591.jpg,"[[17, 71], [19, 95], [23, 119], [30, 142], [38...",0.099748,0.274136,0.290801,0.161706,0.151480,0.163257,0.301005,0.290009,0.072489,0.083641,0.050185,"(9, 29]",1
2466,19,Asian,Male,images\img2592.jpg,"[[10, 60], [12, 86], [15, 113], [18, 139], [28...",0.085143,0.229515,0.263669,0.163613,0.163613,0.204031,0.402965,0.288889,0.076310,0.083202,0.055587,"(9, 29]",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5932,25,White,Female,images\img6207.jpg,"[[3, 49], [3, 77], [6, 105], [11, 131], [22, 1...",0.102729,0.216319,0.271118,0.139753,0.128412,0.195467,0.347131,0.303214,0.084963,0.117535,0.072892,"(9, 29]",1
5933,25,White,Female,images\img6208.jpg,"[[21, 75], [23, 98], [26, 121], [33, 143], [44...",0.083505,0.219746,0.290306,0.136320,0.148088,0.231232,0.489249,0.295233,0.064951,0.105130,0.071102,"(9, 29]",1
5934,25,White,Female,images\img6209.jpg,"[[-10, 58], [-9, 85], [-6, 113], [-1, 140], [1...",0.096998,0.226113,0.285001,0.132574,0.137176,0.155926,0.322458,0.292004,0.091629,0.114314,0.068826,"(9, 29]",1
5935,25,White,Female,images\img6210.jpg,"[[21, 73], [22, 96], [27, 120], [32, 142], [41...",0.115833,0.202963,0.300397,0.153601,0.159617,0.167629,0.340836,0.319471,0.073670,0.134642,0.092902,"(9, 29]",1


Split evenly by ethnicity and age



create bins
child (0-)


In [68]:
#Split the data into train test

#train, test = train_test_split(face_recog, test_size=0.2)

**Use Neural Net to Predict Age**

In [73]:
#Use the encoded age for train and test labels
# train_label = train["age_encoded"].values
# test_label = test["age_encoded"].values
# test_bins = test["age_bins"].values


In [74]:
#These are the bins for age
test_bins

[(9, 29], (59, 69], (0, 9], (9, 29], (29, 39], ..., (9, 29], (9, 29], (9, 29], (49, 59], (29, 39]]
Length: 4505
Categories (10, interval[int64]): [(0, 9] < (9, 29] < (29, 39] < (39, 49] ... (69, 79] < (79, 89] <
                                   (89, 99] < (99, 130]]

In [75]:
#drop all the unused columns for train and test data
# train = train.drop(columns= ["age", "path", "landmarks", "age_bins", "age_encoded"])
# test = test.drop(columns= ["age", "path", "landmarks", "age_bins", "age_encoded"])


In [76]:
train

,age,nose2lip,lip2chin,len_between_eyes,left_eyelid_len,right_eyelid_len,nose_width,size_of_mouth,nose_len,bottom_lip_width,left_eye_size,right_eye_size,ethnicity_encoded,gender_encoded
20084,63,0.118583,0.260488,0.276091,0.127564,0.129357,0.161458,0.411590,0.277107,0.060951,0.102172,0.072881,4,0
7929,26,0.107224,0.268113,0.281439,0.155567,0.160925,0.239580,0.453119,0.284142,0.069695,0.098855,0.058973,3,1
190,1,0.092160,0.334102,0.250039,0.201667,0.195184,0.168180,0.286937,0.245543,0.072379,0.086969,0.050108,2,1
14119,38,0.083898,0.238472,0.228510,0.172864,0.167796,0.211298,0.333699,0.299321,0.077991,0.099002,0.062060,1,0
3796,22,0.089507,0.219941,0.201083,0.183903,0.161082,0.209341,0.365939,0.300297,0.079676,0.106611,0.063182,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6526,26,0.102481,0.231217,0.274578,0.134103,0.139570,0.196867,0.466294,0.268205,0.075289,0.099778,0.065257,2,0
3525,21,0.124795,0.230517,0.277653,0.151100,0.163382,0.222014,0.416295,0.279107,0.083715,0.102606,0.067896,1,1
9831,29,0.120724,0.285658,0.245527,0.198933,0.192113,0.181430,0.431463,0.275088,0.062284,0.091827,0.062284,1,0
6239,26,0.079276,0.225554,0.262357,0.165313,0.187328,0.212762,0.480051,0.282631,0.058659,0.105581,0.075415,1,0


In [77]:
#encode age bins
face_recog["quantiles"] = pd.qcut(face_recog["age"],q=5)
encoder_a = LabelEncoder()
encoder_a.fit(face_recog["quantiles"])
a_train = encoder_a.transform(face_recog["quantiles"])
face_recog.insert(2,"bin",a_train)

In [78]:
age_train,age_test = train_test_split(face_recog,test_size=.3)

In [79]:
age_train_labels = age_train["bin"]
age_train_data = age_train.drop(columns=["bin","path","age","landmarks","age_bins","age_encoded","quantiles"])
age_test_labels = age_test["bin"]
age_test_data = age_test.drop(columns=["bin","path","age","landmarks","age_bins","age_encoded","quantiles"])

In [80]:
#encode ethnicity and gender variables for train data
encoder_e = LabelEncoder()
encoder_e.fit(age_train_data["ethnicity"])
encoder_e.classes_

encoder_g = LabelEncoder()
encoder_g.fit(age_train_data["gender"])
encoder_g.classes_

e_train = encoder_e.transform(age_train_data["ethnicity"])
g_train = encoder_g.transform(age_train_data["gender"])
age_train_data.insert(13, "ethnicity_encoded", e_train)
age_train_data.insert(14, "gender_encoded", g_train)
age_train_data = age_train_data.drop(columns= ["ethnicity", "gender"])


In [81]:
age_train_data.head()

,nose2lip,lip2chin,len_between_eyes,left_eyelid_len,right_eyelid_len,nose_width,size_of_mouth,nose_len,bottom_lip_width,left_eye_size,right_eye_size,ethnicity_encoded,gender_encoded
6338,0.076475,0.207127,0.271261,0.140091,0.144097,0.213020,0.372319,0.325784,0.093976,0.107788,0.082404,3,0
10154,0.116367,0.259120,0.281989,0.175415,0.168623,0.167864,0.301401,0.255179,0.058605,0.097836,0.074282,4,1
14817,0.121040,0.272701,0.248781,0.146573,0.143726,0.178306,0.436457,0.265018,0.056655,0.131607,0.090374,4,0
5630,0.118244,0.285750,0.247314,0.115286,0.098011,0.214409,0.417418,0.299631,0.074784,0.092485,0.046243,4,1
11193,0.085124,0.219102,0.279505,0.118635,0.148616,0.209698,0.414197,0.306920,0.088534,0.106405,0.071073,1,0


In [82]:
#encode ethnicity and gender variables for train data
encoder_e = LabelEncoder()
encoder_e.fit(age_test_data["ethnicity"])
encoder_e.classes_

encoder_g = LabelEncoder()
encoder_g.fit(age_test_data["gender"])
encoder_g.classes_

e_test = encoder_e.transform(age_test_data["ethnicity"])
g_test = encoder_g.transform(age_test_data["gender"])
age_test_data.insert(13, "ethnicity_encoded", e_test)
age_test_data.insert(14, "gender_encoded", g_test)
age_test_data = age_test_data.drop(columns= ["ethnicity", "gender"])

In [83]:
age_train_data

,nose2lip,lip2chin,len_between_eyes,left_eyelid_len,right_eyelid_len,nose_width,size_of_mouth,nose_len,bottom_lip_width,left_eye_size,right_eye_size,ethnicity_encoded,gender_encoded
6338,0.076475,0.207127,0.271261,0.140091,0.144097,0.213020,0.372319,0.325784,0.093976,0.107788,0.082404,3,0
10154,0.116367,0.259120,0.281989,0.175415,0.168623,0.167864,0.301401,0.255179,0.058605,0.097836,0.074282,4,1
14817,0.121040,0.272701,0.248781,0.146573,0.143726,0.178306,0.436457,0.265018,0.056655,0.131607,0.090374,4,0
5630,0.118244,0.285750,0.247314,0.115286,0.098011,0.214409,0.417418,0.299631,0.074784,0.092485,0.046243,4,1
11193,0.085124,0.219102,0.279505,0.118635,0.148616,0.209698,0.414197,0.306920,0.088534,0.106405,0.071073,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9587,0.118741,0.273529,0.271092,0.168747,0.176529,0.189642,0.370472,0.273883,0.064028,0.078946,0.047659,0,1
1165,0.105807,0.223880,0.261505,0.200726,0.189288,0.192713,0.458711,0.266577,0.069296,0.078703,0.031983,2,1
4161,0.105278,0.274082,0.255187,0.137997,0.142599,0.214246,0.504891,0.290520,0.060286,0.074543,0.044186,2,1
4218,0.095238,0.263561,0.276629,0.154343,0.172795,0.180408,0.372049,0.301331,0.068414,0.074077,0.034559,0,1


In [84]:
#run a neural net 

clf = MLPClassifier(hidden_layer_sizes=(10, 10), max_iter=200, alpha=0.001,
                     solver='lbfgs', verbose=10,tol=0.000000001)
clf.fit(age_train_data, age_train_labels)
pred = clf.predict(age_test_data)
print(accuracy_score(age_test_labels, pred))

0.4187629476176384


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Since the accuracy is really low, we decided to look more into different methods to differentiate age. Furthermore, with the additional features, I think we can run a neural net to predict ethnicity. We can probably find better results from predicting ethnicity. 


In [85]:
#This shows you the best variables to use for the model
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

bestfeatures = SelectKBest(score_func=chi2, k=13)
fit = bestfeatures.fit(age_train_data, age_train_labels)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(age_train_data.columns)

featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  #naming the dataframe columns
print(featureScores.nlargest(20,'Score'))

                Specs       Score
11  ethnicity_encoded  846.163040
12     gender_encoded  312.652177
10     right_eye_size   13.061409
1            lip2chin   12.682937
6       size_of_mouth   11.817091
9       left_eye_size   11.106994
8    bottom_lip_width    8.398308
5          nose_width    6.485093
3     left_eyelid_len    5.219456
4    right_eyelid_len    4.791511
0            nose2lip    3.496907
7            nose_len    0.701589
2    len_between_eyes    0.086293


In [86]:
pd.DataFrame({"Predictions":pred, "Actual":age_test_labels, "Bins": test_bins})

ValueError: ignored

**Using Neural Net to Predict ethnicity**

In [ ]:
import random
random.seed(20)
train, test = train_test_split(face_recog, test_size=0.2)

In [ ]:
train = train.drop(columns= ["path", "landmarks", "age_bins", "age_encoded"])
test = test.drop(columns= ["path", "landmarks", "age_bins", "age_encoded"])


In [ ]:
train

In [ ]:
encoder_e = LabelEncoder()
encoder_e.fit(train["ethnicity"])
encoder_e.classes_

encoder_g = LabelEncoder()
encoder_g.fit(train["gender"])
encoder_g.classes_

e_train = encoder_e.transform(train["ethnicity"])
g_train = encoder_g.transform(train["gender"])
train.insert(14, "ethnicity_encoded", e_train)
train.insert(15, "gender_encoded", g_train)
train = train.drop(columns= ["ethnicity", "gender"])



In [ ]:
encoder_e = LabelEncoder()
encoder_e.fit(test["ethnicity"])
encoder_e.classes_

encoder_g = LabelEncoder()
encoder_g.fit(test["gender"])
encoder_g.classes_

e_test = encoder_e.transform(test["ethnicity"])
g_test = encoder_g.transform(test["gender"])
test.insert(14, "ethnicity_encoded", e_test)
test.insert(15, "gender_encoded", g_test)
test = test.drop(columns= ["ethnicity", "gender"])

In [ ]:
train_label = train["ethnicity_encoded"].values
test_label = test["ethnicity_encoded"].values



In [ ]:
train_label

In [ ]:
train.drop(columns = ["ethnicity_encoded"])
test.drop(columns=["ethnicity_encoded"])

In [ ]:
clf = MLPClassifier(hidden_layer_sizes=(20), max_iter=100, alpha=0.0001,
                     solver='lbfgs', verbose=10,  random_state=21,tol=0.000000001)
clf.fit(train, train_label)
pred = clf.predict(test)
print(accuracy_score(test_label, pred))

In [ ]:
df_ethnicity = pd.DataFrame({"Predicted":pred, "Actual" :test_label})
df_ethnicity

In [ ]:
#This shows you the best variables to use for the model
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

bestfeatures = SelectKBest(score_func=chi2, k=13)
fit = bestfeatures.fit(train, train_label)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(train.columns)

featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  #naming the dataframe columns
print(featureScores.nlargest(20,'Score'))